In [1]:
import pandas as pd
#import numpy as np
import os
#import sys
#import matplotlib.pyplot as plt
#import matplotlib
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
from datetime import datetime

In [2]:
driver = webdriver.Chrome('chromedriver')

#### 0. 환경설정

In [3]:
car_domestic = 'http://www.encar.com/dc/dc_carsearchlist.do?carType=kor&searchType=model&TG.R=A#!'
car_foreign = 'http://www.encar.com/fc/fc_carsearchlist.do?carType=for&searchType=model&TG.R=B#!'
car_ev = 'http://www.encar.com/ev/ev_carsearchlist.do?carType=ev&searchType=model&TG.R=D#!'
page_num = 100   # 수집 페이지 수
delay = 1.5   # 딜레이 시간

#### 1. 초기화

In [4]:
# 접속하기
driver.get(car_ev)

# [20개씩 보기] → [50개씩 보기]로 변환
viewer = Select(driver.find_element_by_css_selector('select#pagerow'))
viewer.select_by_value('50')
time.sleep(delay)

# 차종 카테고리 열기
driver.find_element_by_css_selector('div.schset.category').find_element_by_css_selector('h5').click()
time.sleep(delay)

#### 2. 페이지 순회 함수

In [5]:
# 페이지 순회
def get_data_batch(car_category, ev_type):
    stack = []
    page = 1
    while(True):
        rows = driver.find_element_by_css_selector('tbody#sr_normal').find_elements_by_tag_name('tr')
        batch = []
        for row in rows:
            batch.append(row.get_attribute('outerHTML'))
        stack += batch
        try:
            driver.find_element_by_css_selector('div#pagination').find_element_by_xpath('//a[@data-page="{}"]'.format(page)).click()
        except:
            break
        page += 1
        time.sleep(delay)
    df = pd.DataFrame(stack)
    df.columns = ['데이터']
    df.insert(0, '종류2', ev_type)
    df.insert(0, '종류1', car_category)
    return df

#### 3. 순회 시작

In [ ]:
result = []
# 차종 선택
for category_num in range(12):
    if driver.find_element_by_css_selector('input#category_{}'.format(category_num)).find_element_by_xpath('./following-sibling::em').text != '0':
        driver.find_element_by_css_selector('input#category_{}'.format(category_num)).click()
        time.sleep(delay)
        car_category = driver.find_element_by_css_selector('input#category_{}'.format(category_num)).find_element_by_xpath('./following-sibling::label').text
        
    
        # 전기ㆍ친환경차 종류 선택
        for ev_type_num in range(4):
            if driver.find_element_by_css_selector('input#evType_{}'.format(ev_type_num)).find_element_by_xpath('./following-sibling::em').text != '0':
                driver.find_element_by_css_selector('input#evType_{}'.format(ev_type_num)).click()
                time.sleep(delay)
                ev_type = driver.find_element_by_css_selector('input#evType_{}'.format(ev_type_num)).find_element_by_xpath('./following-sibling::label').text
                ################
                print('수집시작(차종: {}, 종류: {})'.format(car_category, ev_type))
                temp = get_data_batch(car_category, ev_type)
                result.append(temp)
                print('수집완료(개수: {}개)'.format(len(temp)))
                ################
                driver.find_element_by_css_selector('input#evType_{}'.format(ev_type_num)).click()
                time.sleep(delay)

        driver.find_element_by_css_selector('input#category_{}'.format(category_num)).click()
        time.sleep(delay)

수집시작(차종: 경차, 종류: 전기차)
수집완료(개수: 156개)
수집시작(차종: 경차, 종류: LPG)
수집완료(개수: 225개)
수집시작(차종: 소형차, 종류: 하이브리드)
수집완료(개수: 40개)
수집시작(차종: 소형차, 종류: 전기차)
수집완료(개수: 76개)
수집시작(차종: 준중형차, 종류: 하이브리드)
수집완료(개수: 360개)
수집시작(차종: 준중형차, 종류: 플러그인 하이브리드)
수집완료(개수: 30개)
수집시작(차종: 준중형차, 종류: 전기차)
수집완료(개수: 228개)
수집시작(차종: 준중형차, 종류: LPG)
수집완료(개수: 188개)
수집시작(차종: 중형차, 종류: 하이브리드)
수집완료(개수: 550개)
수집시작(차종: 중형차, 종류: 플러그인 하이브리드)
수집완료(개수: 50개)
수집시작(차종: 중형차, 종류: 전기차)
수집완료(개수: 50개)
수집시작(차종: 중형차, 종류: LPG)
수집완료(개수: 50개)


In [5]:
# result 폴더 생성
if not any([s == 'result' for s in os.listdir('.')]):
    os.mkdir('result')

# 결과 저장하기
df = pd.concat(result)
today = datetime.now().strftime('%Y%m%d%H%M%S')

writer = pd.ExcelWriter('./result/skencar_carlist_env_{}.xlsx'.format(today))
df.to_excel(writer, index=False)
writer.save()
writer.close()